In [4]:
import numpy as np
import pandas as pd
import ast 

# For scraping
import requests
from bs4 import BeautifulSoup

# For adding delays so that we don't spam requests
import time

import string

import re
import json

In [5]:
def get_genre(str_dic):
    genre_list=[]
    for i in str_dic:
        if i == '"genre":':
            j=str_dic.index(i)+1
            while str_dic[j] not in ['],','"actor":','"director":','"creator":','"datePublished":','}</script>]']:
                for punctuation_mark in string.punctuation:
                    str_dic[j]=str_dic[j].replace(punctuation_mark,'')
                if str_dic[j]!="":   
                    genre_list.append(str_dic[j])
                j+=1
            break  

    return genre_list

In [6]:
df_eighties=pd.read_csv('df_eighties.csv')
df_eighties

,tconst,primaryTitle,startYear
0,tt0015724,Dama de noche,1993
1,tt0036606,"Another Time, Another Place",1983
2,tt0038687,Let There Be Light,1980
3,tt0059900,"Wenn du groß bist, lieber Adam",1990
4,tt0061876,The Commissar,1987
...,...,...,...
26265,tt9673442,Blood and Musk,1997
26266,tt9686178,The Best Years Ever,1994
26267,tt9779906,Big Wave,1984
26268,tt9828802,Vaya luna de miel,1980


In [7]:
scrap_df = pd.DataFrame(columns=['tconst', 'genre', 'plot'])
i=0

for tconst in df_eighties['tconst']:
    
    imdb_to_scrap = {'tconst':"", 'genre':[], 'plot':""}
    
    response2=requests.get(f'https://www.imdb.com/title/{tconst}/plotsummary')
    soup2 = BeautifulSoup(response2.content,'html.parser')
        
    if soup2.title.text.strip() != '404 Error - IMDb':
        plot=soup2.find('ul', id="plot-synopsis-content").text.strip()


        if plot!='It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.':

            imdb_to_scrap = {'tconst':tconst, 'genre':[], 'plot':plot}

            # Send a get request and assign the response to a variable
            response1 = requests.get(f'https://www.imdb.com/title/{tconst}/')
            soup1 = BeautifulSoup(response1.content,'html.parser')


            str_dic=str(soup1.findAll('script', {'type':'application/ld+json'})).split()
            imdb_to_scrap['genre']=[get_genre(str_dic)]


            df = pd.DataFrame.from_dict(imdb_to_scrap)
            scrap_df = scrap_df.append(df, ignore_index=True)
            

            #follow progress: print tconst every 25 iteration
            i+=1
            if i%25==0:
                print(tconst)
            
            


scrap_df

tt0080556
tt0081070
tt0081611
tt0082175
tt0082495
tt0082894
tt0083590
tt0083943
tt0084522
tt0084867
tt0085694
tt0086250
tt0086837
tt0087363
tt0087928
tt0088272
tt0089013
tt0089560
tt0090021
tt0090633
tt0091167
tt0091557
tt0092106
tt0092804
tt0093409
tt0093886
tt0094715
tt0095082
tt0095647
tt0096217
tt0096869
tt0097366
tt0097812
tt0098439
tt0099365
tt0099939
tt0100477
tt0101504
tt0102004
tt0102536
tt0103110
tt0104155
tt0104940
tt0105488
tt0106611
tt0107286
tt0108052
tt0109117
tt0109831
tt0110546
tt0111280
tt0112364
tt0113117
tt0113540
tt0114538
tt0115685
tt0116308
tt0116931
tt0117866
tt0118689
tt0119080
tt0119567
tt0120102
tt0120601
tt0120768
tt0120885
tt0124189
tt0129167
tt0134273
tt0141824
tt0149397
tt0157201
tt0166896
tt0175880
tt0181712
tt0191043
tt0206851
tt0229414
tt0246825
tt0265579
tt0298842
tt0344966
tt0440199
tt1392100


,tconst,genre,plot
0,tt0076276,"[Adventure, Comedy, Drama, War]","On April 5, 1941(one day before the German arm..."
1,tt0078868,"[Drama, contentRating, Not, Rated]",Drama centering on a romance and relationship ...
2,tt0078926,[Comedy],Caleb lives in Galilee at the time of Jesus an...
3,tt0078967,[],El Jato (Manuel de Benito) y El Muertes (Ángel...
4,tt0079007,"[Drama, History]","El 21 de agosto de 1910, en el pueblo de Osa d..."
...,...,...,...
2106,tt2354049,"[Drama, contentRating, Not, Rated]",In this poignant adaptation of James Baldwin's...
2107,tt2379514,[Comedy],Kiran(Naresh) is a manager in a shop of electr...
2108,tt4744132,"[Drama, Family, Romance]",Deepa Nandy lives in Taramati with her sister ...
2109,tt5468886,"[Documentary, contentRating, Not, Rated]","Los Sures introduces itself as ""a portrait of ..."


In [8]:
scrap_df.to_csv ('scraper_df_eighties.csv', index = False, header=True)